Import Libraries

In [0]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys

import keras
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPool1D, CuDNNLSTM, Reshape
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate
from keras.datasets import cifar10
from keras import regularizers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
import keras.backend as K
from keras.objectives import mean_squared_error
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, RobustScaler, StandardScaler

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

Load Data

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
dict = {0:'Airplane', 1:'Automobile', 2:'Bird', 3:'Cat', 4:'Deer', 5:'Dog', 6:'Frog', 7:'Horse', 8:'Ship', 9:'Truck'}

Mujtaba's Augmentation Cifar-10

In [0]:
import tensorflow as tf
import numpy as np                                
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
#load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols , channels= 32,32,3
print('Original images examples')
for i in range(0,9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train[i])
plt.show()

# Number of rows of the dataset
number_of_rows = x_train.shape[0]

# set up image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    )
datagen.fit(x_train)

# see example augmentation images
print('Augmentated images examples')
for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size = number_of_rows):
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i].astype(np.uint8))
    plt.show()
    break

# Combine original with augmentated
x_train_aug = np.concatenate((x_train, X_batch.astype(np.uint8)), axis=0)
y_train_aug = np.concatenate((y_train, y_batch.astype(np.uint8)), axis=0)

class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
num_classes = len(class_names)

x_train_aug = x_train_aug.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train_aug = keras.utils.to_categorical(y_train_aug, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Select 10% randomly from training data
new_size = int(number_of_rows*0.1)
random_indices = np.random.choice(number_of_rows, size = new_size, replace=False)
x_train_random = x_train[random_indices, :]
y_train_random = y_train[random_indices, :]

datagen.fit(x_train_random)

# see example of randomly augmentation images
print('Randomly augmentated images examples')
for X_batch_random, y_batch_random in datagen.flow(x_train_random, y_train_random, batch_size=new_size):
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch_random[i].astype(np.uint8))
    plt.show()
    break

# Combine original with augmentated
x_train_random_aug = np.concatenate((x_train_random, X_batch_random.astype(np.uint8)), axis=0)
y_train_random_aug = np.concatenate((y_train_random, y_batch_random.astype(np.uint8)), axis=0)

x_train_random_aug = x_train_random_aug.astype('float32') / 255.

y_train_random_aug = keras.utils.to_categorical(y_train_random_aug, num_classes)

Creating imbalanced data

In [0]:
''' #use mujtaba's output instead later
x_test_extra = []
y_test_extra = []
x_train_final = []
y_train_final = []
count = [0, 0, 0]
for i, j in zip(x_train, y_train):
    if (j==2):
        if(count[0]<2000):
            x_test_extra.append(i)
            y_test_extra.append(j)
            count[0]+=1
        else:
            x_train_final.append(i)
            y_train_final.append(j)
    elif (j==4):
        if(count[1]<2000):
            x_test_extra.append(i)
            y_test_extra.append(j)
            count[1]+=1
        else:
            x_train_final.append(i)
            y_train_final.append(j)
    elif (j==9):
        if(count[2]<2000):
            x_test_extra.append(i)
            y_test_extra.append(j)
            count[2]+=1
        else:
            x_train_final.append(i)
            y_train_final.append(j)
    else:
        x_train_final.append(i)
        y_train_final.append(j)
        
x_test_extra = np.array(x_test_extra)
y_test_extra = np.array(y_test_extra)
x_train_final = np.array(x_train_final)
y_train_final = np.array(y_train_final) '''

In [0]:
''' x_test_final = np.append(x_test_extra, x_test, axis=0)
y_test_final = np.append(y_test_extra, y_test, axis=0) '''

Data Normalization

In [0]:
''' x_train_final = x_train_final.astype('float32')
x_test_final = x_test_final.astype('float32')
x_train_final = x_train_final / 255
x_test_final = x_test_final / 255 '''

Validation Split

In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_trainf, y_validf = train_test_split(x_train_random_aug, y_train_random_aug, test_size=0.2, random_state=42, shuffle= True)
# x_train_random_aug -> x_train_final, y_train_random_aug -> y_train_final

Target conversion to categorical

In [0]:
y_train = keras.utils.to_categorical(y_trainf, 10)
y_valid = keras.utils.to_categorical(y_validf, 10)
y_test_one_hot = y_train_random_aug #keras.utils.to_categorical(y_train_random_aug, 10)
# y_train_random_aug -> y_test_final 

Necessary functions

In [0]:
def create_block(input, chs): ## Convolution block of 2 layers
    x = input
    for i in range(2):
        x = Conv2D(chs, 3, padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
    return x

##############################

## Here, I compute the class weights for using in different models. 
## This is to order our model to emphasize more on classes with less training data.
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(y_trainf), 
                y_trainf.reshape(y_trainf.shape[0]))

class_weights

##############################

def showOrigDec(orig, dec, num=10):  ## function used for visualizing original and reconstructed images of the autoencoder model
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[300*i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[300*i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
        
def show_test(m, d):  ## function used for visualizing the predicted and true labels of test data
    plt.figure(figsize =(40,8))
    for i in range(5):
        ax = plt.subplot(1, 5, i+1)
        test_image = np.expand_dims(d[1810*i+5], axis=0)
        test_result = m.predict(test_image)
        plt.imshow(x_test_final[1810*i+5])
        index = np.argsort(test_result[0,:])
        plt.title("Pred:{}, True:{}".format(dict[index[9]], dict[y_train_random_aug[1810*i+5][0]]))
        # y_train_random_aug -> y_test_final
    plt.show()
    
def report(predictions): ## function used for creating a classification report and confusion matrix
    cm=confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
    print("Classification Report:\n")
    cr=classification_report(y_test.argmax(axis=1),
                                predictions.argmax(axis=1), 
                                target_names=list(dict.values()))
    print(cr)
    plt.figure(figsize=(12,12))
    sns.heatmap(cm, annot=True, xticklabels = list(dict.values()), yticklabels = list(dict.values()), fmt="d")

Autoencoder Model

In [0]:
def general_ae():
    input = Input((32,32,3))
    
    # Encoder
    block1 = create_block(input, 32)
    x = MaxPool2D(2)(block1)
    block2 = create_block(x, 64)
    
    #Middle
    x = MaxPool2D(2)(block2)
    middle = create_block(x, 128)
    
    # Decoder
    block3 = create_block(middle, 64)
    up1 = UpSampling2D((2,2))(block3)
    block4 = create_block(up1, 32)
    up2 = UpSampling2D((2,2))(block4)
    
    # output
    x = Conv2D(3, 1)(up2)
    output = Activation("sigmoid")(x)
    return Model(input, middle), Model(input, output)

In [0]:
def run_ae(m):  ## function for Implementing general autoencoder
    if m=='ae':
        encoder, model = general_ae()
        
    return encoder, model

In [0]:
def loss_function(y_true, y_pred):  ## loss function for using in autoencoder models
    mses = mean_squared_error(y_true, y_pred)
    return K.sum(mses, axis=(1,2))

Implementing AE

In [0]:
encoder_ae, model_ae = run_ae('ae')
model_ae.compile(SGD(1e-3, 0.9), loss=loss_function)
model_ae.summary()

In [0]:
er = EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True)
lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_delta=0.0001)
callbacks = [er, lr]
history = model_ae.fit(x_train, x_train, 
                       batch_size=512,
                       epochs=100,
                       verbose=1,
                       validation_data=(x_valid, x_valid),
                       shuffle=True, callbacks=callbacks,
                       class_weight=class_weights)

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [0]:
recon_test_ae = model_ae.predict(x_test)
recon_valid_ae = model_ae.predict(x_valid)

In [0]:
showOrigDec(x_valid, recon_valid_ae)

In [0]:
showOrigDec(x_test, recon_test_ae)

Extracting bottleneck features to use as inputs in the classifier model

In [0]:
gist_train_ae = encoder_ae.predict(x_train)
gist_valid_ae = encoder_ae.predict(x_valid)
gist_test_ae = encoder_ae.predict(x_test)

Classifier Models

In [0]:
def classifier_conv(inp):
    input = Input((inp.shape[1], inp.shape[2], inp.shape[3]))
    x = Conv2D(1024, 3, padding="same")(input)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(2)(x)
    x = Dropout(0.5)(x)
    x = Conv2D(128, 3, padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(2)(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.35)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.69)(x)
    output = Dense(10, activation='softmax')(x)
    return Model(input, output)

In [0]:
def run_cls(m, inp):  ## function for Implementing convolutional classifier model
    if m=='conv':
        classifier = classifier_conv(inp)
        
    return classifier

Convolutional AE with convolutional NN as classifier

In [0]:
decoder_ae_conv = run_cls('conv', gist_train_ae)
decoder_ae_conv.compile(loss='categorical_crossentropy',
                        optimizer=Adadelta(),
                        metrics=['accuracy'])
decoder_ae_conv.summary()

In [0]:
er = EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True)
lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_delta=0.0001)
callbacks = [er, lr]
hist1 = decoder_ae_conv.fit(gist_train_ae, y_trainf, batch_size=512, epochs=100, 
                            validation_data = (gist_valid_ae, y_valid),
                            shuffle=True, callbacks=callbacks,
                            class_weight=class_weights)

In [0]:
plt.plot(hist1.history['accuracy'])
plt.plot(hist1.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [0]:
print('Test accuracy for AE_conv model= {}'.format(decoder_ae_conv.evaluate(gist_test_ae, y_test)[1]))

In [0]:
show_test(decoder_ae_conv, gist_test_ae)

In [0]:
predictions = decoder_ae_conv.predict(gist_test_ae)
report(predictions)